In [5]:
#v3 quitando la info de orden de movimientos mediante el metodo NormalizarTablero(), (0-"vacio", 1-"X", 2-"O")
#A ver si esa informacion adicional (pero innecesaria) confundia la Red
#La etiqueta continua igual. La misma puede ser 0, 1 o 2   (0-Empate, 1-ganó X, 2-ganó O)
#adicionalmente, randomizamos el set de datos de train vs test

#RESULTADOS TESTEO 
#
#MODELO v2  maximo fue 0.82, hicieron falta 4 capas y mucha prueba y error
#
#MODELO V3
#Con 2 capas (9 y 3)
#  Epochs  |   Batch   | Score
#  5,5,5   | 128,64,32 | 0.8628
#  15,0,0  | 128,64,32 | 0.83
#  0,15,0  | 128,64,32 | 0.8942
#  0,0,15  | 128,64,32 | 0.8427
#  0,10,5  | 128,64,32 | 0.901
#  0,10,10 | 128,64,32 | 0.944

#Con 3 capas (9,9 y 3)
#  0,10,5  | 128,64,32 | 0.979
#  0,10,10 | 128,64,32 | 0.984

#Con 4 capas (9,9, 9 y 3)
#  0,5,0  | 128,64,32 | 0.974
#  0,5,5  | 128,64,32 | 0.994
#  10,0,0 | 128,64,32 | 0.998   (USANDO ESTA!)
#  0,10,0 | 128,64,32 | 0.971



In [ ]:
import numpy as np
from copy import copy, deepcopy
import random
import datetime

listaPartidas = []  
listaEtiquetas = []

def HayGanador(tablero):
    #Recibe un Tablero,
    #retorna:   0 si nadie esta ganando, 1 si cruz y 2 si circulo
    result = 0
    
    #filas
    if (not(tablero[0,0]==0 or tablero[0,1]==0 or tablero[0,2]==0)) and (tablero[0,0]%2 == tablero[0,1]%2 == tablero[0,2]%2 == 1):
        result = 1
    if (not(tablero[0,0]==0 or tablero[0,1]==0 or tablero[0,2]==0)) and (tablero[0,0]%2 == tablero[0,1]%2 == tablero[0,2]%2 == 0):
        result = 2       
    if (not(tablero[1,0]==0 or tablero[1,1]==0 or tablero[1,2]==0)) and (tablero[1,0]%2 == tablero[1,1]%2 == tablero[1,2]%2 == 1): 
        result = 1
    if (not(tablero[1,0]==0 or tablero[1,1]==0 or tablero[1,2]==0)) and (tablero[1,0]%2 == tablero[1,1]%2 == tablero[1,2]%2 == 0): 
        result = 2     
    if (not(tablero[2,0]==0 or tablero[2,1]==0 or tablero[2,2]==0)) and (tablero[2,0]%2 == tablero[2,1]%2 == tablero[2,2]%2 == 1):
        result = 1
    if (not(tablero[2,0]==0 or tablero[2,1]==0 or tablero[2,2]==0)) and (tablero[2,0]%2 == tablero[2,1]%2 == tablero[2,2]%2 == 0):
        result = 2    
    
    #columnas
    if (not(tablero[0,0]==0 or tablero[1,0]==0 or tablero[2,0]==0)) and (tablero[0,0]%2 == tablero[1,0]%2 == tablero[2,0]%2 == 1):
        result = 1
    if (not(tablero[0,0]==0 or tablero[1,0]==0 or tablero[2,0]==0)) and (tablero[0,0]%2 == tablero[1,0]%2 == tablero[2,0]%2 == 0):
        result = 2       
    if (not(tablero[0,1]==0 or tablero[1,1]==0 or tablero[2,1]==0)) and (tablero[0,1]%2 == tablero[1,1]%2 == tablero[2,1]%2 == 1):
        result = 1
    if (not(tablero[0,1]==0 or tablero[1,1]==0 or tablero[2,1]==0)) and (tablero[0,1]%2 == tablero[1,1]%2 == tablero[2,1]%2 == 0):
        result = 2            
    if (not(tablero[0,2]==0 or tablero[1,2]==0 or tablero[2,2]==0)) and (tablero[0,2]%2 == tablero[1,2]%2 == tablero[2,2]%2 == 1):
        result = 1
    if (not(tablero[0,2]==0 or tablero[1,2]==0 or tablero[2,2]==0)) and (tablero[0,2]%2 == tablero[1,2]%2 == tablero[2,2]%2 == 0):
        result = 2    
        
    #diagonales
    if (not(tablero[0,0]==0 or tablero[1,1]==0 or tablero[2,2]==0)) and (tablero[0,0]%2 == tablero[1,1]%2 == tablero[2,2]%2 == 1):
        result = 1
    if (not(tablero[0,0]==0 or tablero[1,1]==0 or tablero[2,2]==0)) and (tablero[0,0]%2 == tablero[1,1]%2 == tablero[2,2]%2 == 0):
        result = 2         
    if (not(tablero[0,2]==0 or tablero[1,1]==0 or tablero[2,0]==0)) and (tablero[0,2]%2 == tablero[1,1]%2 == tablero[2,0]%2 == 1):
        result = 1    
    if (not(tablero[0,2]==0 or tablero[1,1]==0 or tablero[2,0]==0)) and (tablero[0,2]%2 == tablero[1,1]%2 == tablero[2,0]%2 == 0):
        result = 2    
    
    return result

def SiguienteMovimiento(tablero):
    #devolver max+1
    result = np.max(tablero) + 1
    return result

def NormalizarTablero(tablero):
    #convierte la matriz a 1's (X) y 2's (O)
    
    tableroResult = deepcopy(tablero)
    
    for fila in range(3):
            for columna in range(3):
                if not(tableroResult[fila,columna] == 0):
                    if (tableroResult[fila,columna]%2 == 1):
                        tableroResult[fila,columna] = 1
                    if (tableroResult[fila,columna]%2 == 0):
                        tableroResult[fila,columna] = 2
    return tableroResult               
    
def unison_shuffled_copies(a, b):
    #SEEDING
    from numpy.random import seed
    seed(1)
    from tensorflow import set_random_seed
    set_random_seed(1)
    
    c = list(zip(a, b))
    random.shuffle(c)
    return zip(*c)

def JugarSiguiente(tableroPrevio):  
    tableroResult = tableroPrevio
    global listaPartidas
    global listaEtiquetas
    
    #Algoritmo recursivo
    #Caso Base 1: si alguien ya gano. Agrego al set de datos y retorno
    #Caso Base 2: No hay ganador pero tampoco  hay movimientos posibles. Agrego al set de datos y retorno
    #Si no aplica ningun caso base, tomo un movimiento y hago llamado recursivo
    
    #Caso Base 1: si alguien ya gano
    ganador = HayGanador(tableroResult)
    if (ganador == 1) or (ganador == 2):
        #print(tableroResult)
        listaPartidas.append(NormalizarTablero(tableroResult))
        listaEtiquetas.append(ganador)
        return tableroResult
    
    #Caso Base 2: no quedan movimientos posibles
    if ((tableroResult[tableroResult==0].size) ==0):
        #al no quedar movimientos posibles y no haber ganador, esta partida es empate.
        #print(tableroResult)
        listaPartidas.append(NormalizarTablero(tableroResult))
        listaEtiquetas.append(0)
        return tableroResult
        
    elif (ganador==0):
        for fila in range(3):
            for columna in range(3):
                if tableroResult[fila,columna] == 0:
                    tableroResult[fila,columna] = SiguienteMovimiento(tableroResult)
                    tableroResult = JugarSiguiente(tableroResult)
                    tableroResult[fila,columna] = 0
    
    return tableroResult

def GenerarPartidas():
    global listaPartidas
    global listaEtiquetas
    listaPartidas = []  
    listaEtiquetas = []
    
    #SEEDING
    from numpy.random import seed
    seed(1)
    from tensorflow import set_random_seed
    set_random_seed(1)

    start_time = datetime.datetime.now()
    print("Comenzando a iterar, deberia dar un total de: 255168")
    cuenta = 0
    for fila in range(3):
        for columna in range(3):
            cuenta = cuenta + 1
            print("iteracion " + str(cuenta) + " de 9", end="\r") 
            
            tablero = np.matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
            tablero[fila,columna] = 1
            
            tablero = JugarSiguiente(tablero)
    
    print("")
    print("Se terminaron de generar partidas, total:" + str(len(listaPartidas)))
    print("randomizando datos..")
    (listaPartidas, listaEtiquetas) = unison_shuffled_copies(listaPartidas,listaEtiquetas)
    
    end_time = datetime.datetime.now()
    print("Terminado! en t=" + str(end_time-start_time) + "  (partidas: " +  str(len(listaPartidas)) + ", etiquetas: " + str(len(listaEtiquetas)) + ")" )




In [ ]:
#import numpy as np
#tablero = np.matrix([[1, 2, 0], 
#                     [1, 2, 0], 
#                     [0, 2, 1]])
#HayGanador(tablero)

GenerarPartidas()




Comenzando a iterar, deberia dar un total de: 255168


In [4]:
print(listaPartidas[3645])
print(listaEtiquetas[3645])


[[1 1 2]
 [2 1 1]
 [2 1 2]]
1


In [5]:
start_time = datetime.datetime.now()
end_time = datetime.datetime.now()
print(end_time-start_time)

0:00:00


In [6]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [7]:
import random
import keras

#SEEDING
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

print(len(listaPartidas))
print(len(listaEtiquetas))

#Esta variable de ratio controla que % de datos se usan para entrenamiento, el resto sera validacion
#Por ejemplo, 0.90 quiere decir que del total de datos, el 90% va para train y el 10% para test
ratio_train = 0.10
len_train = round(len(listaPartidas)*ratio_train)


train_set = listaPartidas[1:len_train:1]
train_labels = listaEtiquetas[1:len_train:1]

test_set = listaPartidas[len_train+1:len(listaEtiquetas)-1:1]
test_labels = listaEtiquetas[len_train+1:len(listaEtiquetas)-1:1]


print(len(train_set))
print(len(train_labels))

print(len(test_set))
print(len(test_labels))

255168
255168
25516
25516
229649
229649


In [8]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(9, activation='relu', input_shape=(3 * 3,)))
network.add(layers.Dense(9, activation='relu'))
network.add(layers.Dense(9, activation='relu'))
network.add(layers.Dense(3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [10]:
#ajusto formas de datos a lo especificado en las capas de la red neuronal (shape a 3*3)
#divido los numeros en los datos para que los valores esten todos entre [0,1]

train_set = np.squeeze(np.asarray(train_set))
train_set = train_set.reshape((len(train_set), 3 * 3))
train_set = train_set.astype('float32') / 2

test_set = np.squeeze(np.asarray(test_set))
test_set = test_set.reshape((len(test_set), 3 * 3))
test_set = test_set.astype('float32') / 2

In [11]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)



In [16]:
network.fit(train_set, train_labels, epochs=10, batch_size=16)
#network.fit(train_set, train_labels, epochs=10, batch_size=64)
#network.fit(train_set, train_labels, epochs=5, batch_size=32)

Epoch 1/10
25516/25516 [==============================] - 4s 150us/step - loss: 0.0176 - acc: 0.9958
Epoch 2/10
25516/25516 [==============================] - 4s 148us/step - loss: 0.0154 - acc: 0.9964
Epoch 3/10
25516/25516 [==============================] - 4s 145us/step - loss: 0.0134 - acc: 0.9968
Epoch 4/10
25516/25516 [==============================] - 4s 143us/step - loss: 0.0100 - acc: 0.9974
Epoch 5/10
25516/25516 [==============================] - 4s 141us/step - loss: 0.0079 - acc: 0.9976
Epoch 6/10
25516/25516 [==============================] - 4s 146us/step - loss: 0.0056 - acc: 0.9980
Epoch 7/10
25516/25516 [==============================] - 4s 146us/step - loss: 0.0042 - acc: 0.9982
Epoch 8/10
25516/25516 [==============================] - 4s 153us/step - loss: 0.0025 - acc: 0.9989
Epoch 9/10
25516/25516 [==============================] - 4s 148us/step - loss: 0.0018 - acc: 0.9993
Epoch 10/10
25516/25516 [==============================] - 4s 149us/step - loss: 0.0012 - a

In [ ]:
test_loss, test_acc = network.evaluate(test_set, test_labels)
print('test_acc:', test_acc)